TRABAJO FINAL: Implementación de Algoritmos (PCA, SVD, A-Priori).

          Héctor Jesús Aparicio Muñoz
          Asignatura: Aprendizaje No Supervisado
          Máster Universitario en Inteligencia de Negocio y Big Data en Entornos Seguros

NOTA: la matriz M de entrada de datos debe estar centrada en los algoritmos PCA y SVD (para PCA seguro y casi seguro para SVD). El centrado se hace restando la media. Si la matriz de datos es nxd (n instancias, con d atributos cada una), la media es la media de todas las filas, por tanto un vector 1xd. El vector de medias podría obtenerlo de la muestra, o ser conocido para toda la población, si se estima de la muestra, a la hora de obtener la matriz de covarianzas debería tener el factor corrector 1/(n-1).

## Primeramente importamos la librería Numpy y creamos la función N_frobenius para calcular la Norma de Frobenius de una matriz, ya que lo necesitaremos para el resto de algoritmos:

In [1]:
# Importamos la librería Numpy, ya que vamos a manipular arrays numéricos en los algoritmos que vamos a implementar
import numpy as np

In [2]:
# Función para calcular la norma de Frobenius de un array

def N_frobenius(matrix):
    """Función que calcula la norma de Frobenius de un array.
    
    Parameters
    ----------
    matrix : Array para el cual queremos calcular su norma de Frobenius.
    
    Returns
    -------
    norma : Valor de la norma de Frobenius del array de entrada.
    """
    aux = 0  # Variable auxiliar que utilizaré para calcular la norma de Frobenius del array.
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            aux += (matrix[i][j])**2
    norma = aux**.5
    return norma

## Aclaración del código usado en los algoritmos:

In [3]:
# Ejemplo: queremos montar una matriz de 2x3 a partir de una lista que contiene los vectores columna de dicha
# matriz. Veamos la diferencia entre usar "reshape(2, 3)" o usar "reshape(3, 2).T"

# Creo los siguientes vectores columna
x1 = np.ones((2, 1), dtype=float)
x2 = 4*x1
x3 = 9*x1

# Creo la lista vacía E a la que iré añadiendo en orden los vectores columna x1, x2, x3
E = []
E.append(x1)
E.append(x2)
E.append(x3)

# Transformamos ahora la lista de vectores columna E en una matriz de 2x3.

# Si calculo la matriz de dimensión 2x3 de esta forma vemos que no obtenemos el resultado deseado
E1 = np.array(E).reshape(2, 3)
print(E)
print(E1)

print("\n")

# Veamos como la matriz 2x3 al calcularla de esta otra forma nos da el resultado que buscábamos
E2 = np.array(E).reshape(3, 2).T
print(E)
print(E2)

[array([[1.],
       [1.]]), array([[4.],
       [4.]]), array([[9.],
       [9.]])]
[[1. 1. 4.]
 [4. 9. 9.]]


[array([[1.],
       [1.]]), array([[4.],
       [4.]]), array([[9.],
       [9.]])]
[[1. 4. 9.]
 [1. 4. 9.]]


## Veamos el primer algoritmo implementado sobre el tema de "Reducción de la Dimensionalidad":
### - Análisis de Componentes Principales (PCA)

In [4]:
# Función que implementa el algoritmo PCA
# Debemos haber importado la librería Numpy con anterioridad, y haber definido la función N_frobenius.

def PCA_algoritmo(M, k, converg):
    """Función que implementa el método de análisis de componentes principales
    para reducción de la dimensionalidad.
    
    Parameters
    ----------
    M       : Matriz original de datos de la cual queremos reducir su dimensionalidad. Su número de filas
              es el número de registros o datos diferentes que contiene, y su número de columnas es el número
              de dimensiones que representan a cada uno de los datos. Será un array de Numpy.
              Debe de estar centrada.
    k       : Número entero, indica el número de dimensiones a las que queremos reducir los registros de la
              matriz original de datos.
    converg : Valor de tipo double, de parada para el bucle que calcula los vectores propios por el método de la
              potencia. Cuando la norma de Frobenius de la diferencia entre el valor estimado para el vector
              propio en dos iteraciones sucesivas del método de la potencia sea menor que converg, paramos la
              iteración y nos quedamos con esa última estimación del vector propio.
    
    Returns
    -------
    Mk : Array de numpy bi-dimensional. Tantas filas como registros tiene la matriz original M,
         tantas columnas como número de dimensiones al que queremos reducir los datos, es decir k.
         Es la representación en k dimensiones de la matriz de datos original M.
    """
    
    # Queremos calcular los vectores propios de la matriz cuadrada  M transpuesta * M
    A = M.T@M
    
    # Utilizamos el método de la potencia para encontrar los pares propios de la matraiz A anterior.
    
    lista_E = []  # Lista vacía en la que iré almacenando los vectores propios que vaya calculando para la
                  # matriz A. Posteriormente la transformaré en la matriz E cuyas columnas son los vectores
                  # propios ordenados de mayor a menor valor propio.
    
    # Iniciamos el método de la potencia con un vector inicial xi cualquiera, siempre que sea distinto de cero.
    # La dimensión de dicho vector será igual a la dimensión de la matriz cuadrada A.
    dim_A = len(A)
    xi = np.ones((dim_A, 1), dtype=float)  # Creamos un vector columna de unos, tantos como dimensión tenga A.
    
    for i in range(k):
        # Como hemos dicho aplicamos el método de la potencia para calcular los pares propios de la matriz A.
        # Para ello aplicamos de manera iterativa la siguiente ecuación: xk+1 = (A*xk)/(||A*xk||), siendo el
        # denominador anterior la norma de Frobenius de A*xk .
        # Además este método me los va dando ordenados de mayor a menor valor del autovalor como nos interesa.
        
        num = A@xi  # Ese producto es un vector.
        den = N_frobenius(num)
        xn = num/den  # Nuevo valor del vector propio.
        
        diff = N_frobenius(xi - xn)  # Calculo la norma del vector difrencia entre el vector inicial y el
                                     # obtenido de la primera iteración.
        # Seguiremos iterando hasta que dicha norma sea menor que el parámetro "converg" de convergencia
        while diff >= converg:
            xi = xn
            num = A@xi
            den = N_frobenius(num)
            xn = num/den
            diff = N_frobenius(xi - xn)
        
        # Una vez que tenemos el vector propio calculado, lo añadimos a la lista E
        lista_E.append(xn)
        
        # Calcularemos tantos vectores propios como dimensiones a las que queremos reducir la matriz original
        # de datos, es decir k
        if i+1 < k:
            # Calculamos el valor propio asociado al vector propio "xn" que acabamos de calcular.
            autovalor = xn.T@A@xn
            # Construímos una nueva matriz A para aplicarle de nuevo el método de la potencia y obtener un nuevo
            # par propio.
            A = A - autovalor*xn@xn.T
            xi = xn
    
    # Transformamos la lista con los vectores propios lista_E en una matriz E en la que dichos vectores propios se
    # ordenarán como vectores columna ordenados de mayor a menor valor propio.
    E = np.array(lista_E).reshape(k, dim_A).T
    
    # La representación en k dimensiones de la matriz de datos original M será la siguiente
    Mk = M@E
    
    return Mk

In [5]:
# Ejemplo de uso del algoritmo PCA

# Dada la matriz M que contiene 5 registros y 10 dimensiones
M = np.array([[3, 0, 0, 7, 4, 7, 8, 0, 0, 0], [1, 1, 0, 5, 0, 0, 8, 3, 0, 2], [0, 2, 2, 6, 6, 1, 1, 0, 5, 0], 
              [4, 0, 2, 7, 9, 4, 1, 1, 2, 1], [0, 0, 2, 2, 0, 0, 8, 4, 5, 6]])

# Le aplicamos el algoritmo PCA para que los 5 registros queden representados con 3 dimensiones
k = 3
Mk = PCA_algoritmo(M, k, 0.01)

print(f"Matriz de datos original: M\n\n{M}\n")
print(f"Representación de {k} dimensiones de la matriz M de datos original: Mk\n\n{Mk}")

Matriz de datos original: M

[[3 0 0 7 4 7 8 0 0 0]
 [1 1 0 5 0 0 8 3 0 2]
 [0 2 2 6 6 1 1 0 5 0]
 [4 0 2 7 9 4 1 1 2 1]
 [0 0 2 2 0 0 8 4 5 6]]

Representación de 3 dimensiones de la matriz M de datos original: Mk

[[12.43294797  0.59571495 -5.49971112]
 [ 8.00176753 -5.37266099 -1.72761605]
 [ 8.15122145  3.86536325  4.31575973]
 [11.14902216  6.47391002  1.26136192]
 [ 8.17024449 -8.01484552  3.70866716]]


## Veamos ahora el otro algoritmo implementado sobre el tema de "Reducción de la Dimensionalidad":
### - Descomposición en Valores Singulares (SVD)

In [6]:
# Función que implementa el algoritmo SVD
# Debemos haber importado la librería Numpy con anterioridad, y haber definido la función N_frobenius.

def SVD_algoritmo(M, r, converg):
    """Función que implementa el método de descomposición en valores singulares
    para reducción de la dimensionalidad.
    
    Parameters
    ----------
    M       : Matriz original de datos de la cual queremos reducir su dimensionalidad. Su número de filas
              es el número de registros o datos diferentes que contiene, y su número de columnas es el número
              de dimensiones que representan a cada uno de los datos. Será un array de Numpy.
              Casi con total seguridad debe de estar centrada.
    r       : Número entero, indica el número de valores singulares con los que nos quedamos para reducir la
              dimensionalidad de las matrices U, Σ y V. Si conocemos el número de "conceptos" subyacentes en
              la matriz M, r equivaldría a ese número de conceptos.
    converg : Valor de tipo double, de parada para el bucle que calcula los vectores propios por el método de la
              potencia. Cuando la norma de Frobenius de la diferencia entre el valor estimado para el vector
              propio en dos iteraciones sucesivas del método de la potencia sea menor que converg, paramos la
              iteración y nos quedamos con esa última estimación del vector propio.
    
    Returns
    -------
    U : Array de numpy bi-dimensional. Tendrá tantas filas como registros tiene la matriz original M,
        y su número de columnas será r. Los vectores columna de U son los vectores singulares izquierdos de M.
    Σ : Array de numpy bi-dimensional. Es una matriz diagonal de dimensión rxr, siendo los elementos de la
        diagonal los valores singulares de M, los cuales son todos no negativos y están ordenados de mayor a menor.
    V : Array de numpy bi-dimensional. Tendrá tantas filas como columnas tiene la matriz original M,
        y su número de columnas será r. Los vectores columna de U son los vectores singulares derechos de M.
    """
    
    # La descomposición SVD de la matriz M es M=U*Σ*V_transpuesta. Debemos calcular esas tres matrices U, Σ y V.
    # Para conseguir reducir la dimensión de esas tres matrices U, Σ y V, nos quedamos con los r valores
    # singulares más grandes, hacemos que el resto sean cero.
    # Se puede demostrar que V es la matriz de vectores propios de la matriz cuadrada y simétrica M_transpuesta*M
    # A su vez, U es la matriz de vectores propios de la matriz cuadrada y simétrica M*M_transpuesta
    # La matriz diagonal Σ la calculamos aplicando raíz cuadrada a la matriz diagonal Σ*Σ cuyos valores son los
    # valores propios correspondientes por ejemplo a los vectores propios de la matriz U o a los vectores propios
    # de la matriz V, que como veremos son los mismos.
    
    # ---------- Cálculo de U -----------------------------
    # Empecemos calculando U, para lo cual debemos calcular los vectores propios de la matriz M * M_transpuesta
    MU = M@M.T
    
    # Utilizamos el método de la potencia para encontrar los pares propios de la matraiz MU anterior.
    
    lista_U = []  # Lista vacía en la que iré almacenando los vectores propios que vaya calculando para la
                  # matriz MU. Posteriormente la transformaré en la matriz U cuyas columnas son los vectores
                  # propios ordenados de mayor a menor valor propio.
    autovalor_MU = []  # Lista vacía para almacenar los autovalores de la matriz MU de mayor a menor.
    
    # Iniciamos el método de la potencia con un vector inicial xi cualquiera, siempre que sea distinto de cero.
    # La dimensión de dicho vector será igual a la dimensión de la matriz cuadrada MU.
    dim_MU = len(MU)
    xi = np.ones((dim_MU, 1), dtype=float)  # Creamos un vector columna de unos, tantos como dimensión tenga MU.
    
    for i in range(r):
        # Como hemos dicho aplicamos el método de la potencia para calcular los pares propios de la matriz MU.
        # Para ello aplicamos de manera iterativa la siguiente ecuación: xk+1 = (MU*xk)/(||MU*xk||), siendo el
        # denominador anterior la norma de Frobenius de MU*xk .
        # Además este método me los va dando ordenados de mayor a menor valor del autovalor como nos interesa.
        
        num = MU@xi  # Ese producto es un vector.
        den = N_frobenius(num)
        xn = num/den  # Nuevo valor del vector propio.
        
        diff = N_frobenius(xi - xn)  # Calculo la norma del vector difrencia entre el vector inicial y el
                                     # obtenido de la primera iteración.
        # Seguiremos iterando hasta que dicha norma sea menor que el parámetro "converg" de convergencia
        while diff >= converg:
            xi = xn
            num = MU@xi
            den = N_frobenius(num)
            xn = num/den
            diff = N_frobenius(xi - xn)
        
        # Una vez que tenemos el vector propio calculado, lo añadimos a la lista U
        lista_U.append(xn)
        # Calculamos el valor propio asociado al vector propio "xn" que acabamos de calcular.
        autovalor = xn.T@MU@xn
        autovalor_MU.append(autovalor)
        
        # Calcularemos tantos vectores propios como conceptos conozcamos de la matriz original M, o como valores
        # singulares nos quedemos, es decir r
        if i+1 < r:
            # Calculamos el valor propio asociado al vector propio "xn" que acabamos de calcular.
            #autovalor = xn.T@MU@xn
            # Construímos una nueva matriz MU para aplicarle de nuevo el método de la potencia y obtener un nuevo
            # par propio.
            MU = MU - autovalor*xn@xn.T
            xi = xn
    
    # Transformamos la lista con los vectores propios lista_U en una matriz U en la que dichos vectores propios se
    # ordenarán como vectores columna ordenados de mayor a menor valor propio.
    U = np.array(lista_U).reshape(r, dim_MU).T
    
    # ---------- Cálculo de V ------------------------------
    # Calculemos ahora V, para lo cual debemos calcular los vectores propios de la matriz M_transpuesta * M
    MV = M.T@M
    
    # Utilizamos el método de la potencia para encontrar los pares propios de la matraiz MV anterior.
    
    lista_V = []  # Lista vacía en la que iré almacenando los vectores propios que vaya calculando para la
                  # matriz MV. Posteriormente la transformaré en la matriz V cuyas columnas son los vectores
                  # propios ordenados de mayor a menor valor propio.
    autovalor_MV = []  # Lista vacía para almacenar los autovalores de la matriz MV de mayor a menor.
    
    # Iniciamos el método de la potencia con un vector inicial xi cualquiera, siempre que sea distinto de cero.
    # La dimensión de dicho vector será igual a la dimensión de la matriz cuadrada MV.
    dim_MV = len(MV)
    xi = np.ones((dim_MV, 1), dtype=float)  # Creamos un vector columna de unos, tantos como dimensión tenga MV.
    
    for i in range(r):
        # Como hemos dicho aplicamos el método de la potencia para calcular los pares propios de la matriz MV.
        # Para ello aplicamos de manera iterativa la siguiente ecuación: xk+1 = (MV*xk)/(||MV*xk||), siendo el
        # denominador anterior la norma de Frobenius de MV*xk .
        # Además este método me los va dando ordenados de mayor a menor valor del autovalor como nos interesa.
        
        num = MV@xi  # Ese producto es un vector.
        den = N_frobenius(num)
        xn = num/den  # Nuevo valor del vector propio.
        
        diff = N_frobenius(xi - xn)  # Calculo la norma del vector difrencia entre el vector inicial y el
                                     # obtenido de la primera iteración.
        # Seguiremos iterando hasta que dicha norma sea menor que el parámetro "converg" de convergencia
        while diff >= converg:
            xi = xn
            num = MV@xi
            den = N_frobenius(num)
            xn = num/den
            diff = N_frobenius(xi - xn)
        
        # Una vez que tenemos el vector propio calculado, lo añadimos a la lista V
        lista_V.append(xn)
        # Calculamos el valor propio asociado al vector propio "xn" que acabamos de calcular.
        autovalor = xn.T@MV@xn
        autovalor_MV.append(autovalor)
        
        # Calcularemos tantos vectores propios como conceptos conozcamos de la matriz original M, o como valores
        # singulares nos quedemos, es decir r
        if i+1 < r:
            # Calculamos el valor propio asociado al vector propio "xn" que acabamos de calcular.
            #autovalor = xn.T@MV@xn
            # Construímos una nueva matriz MV para aplicarle de nuevo el método de la potencia y obtener un nuevo
            # par propio.
            MV = MV - autovalor*xn@xn.T
            xi = xn
    
    # Transformamos la lista con los vectores propios lista_V en una matriz V en la que dichos vectores propios se
    # ordenarán como vectores columna ordenados de mayor a menor valor propio.
    V = np.array(lista_V).reshape(r, dim_MV).T
    
    # ------------------------------------------------------
    
    print("Comprobación de que los autovalores obtenidos con las matrices M * M_transpuesta y M_transpuesta * M \
son iguales")
    print(f"Autovalores de la matriz MU = M * M_transpuesta: {autovalor_MU}")
    print(f"Autovalores de la matriz MV = M_transpuesta * M: {autovalor_MV}\n")
    
    # ---------- Cálculo de Σ ------------------------------
    # Calculamos la matriz Σ, a partir de los autovalores calculados. Sabemos que la matriz Σ*Σ es una matriz
    # diagonal con los autovalores anteriores en la diagonal principal. Por lo tanto la matriz de valores
    # singulares Σ será la misma matriz, pero conteniendo en la diagonal principal la raíz cuadrada de los
    # autovalores.
    
    val_singulares = np.array(autovalor_MV).reshape(1, r)**.5
    Σ = np.diagflat(val_singulares)
    
    return U, Σ, V

In [7]:
# Ejemplo de uso del algoritmo SVD

# Dada la matriz M que contiene 5 registros y 10 dimensiones
M = np.array([[3, 0, 0, 7, 4, 7, 8, 0, 0, 0], [1, 1, 0, 5, 0, 0, 8, 3, 0, 2], [0, 2, 2, 6, 6, 1, 1, 0, 5, 0], 
              [4, 0, 2, 7, 9, 4, 1, 1, 2, 1], [0, 0, 2, 2, 0, 0, 8, 4, 5, 6]])

# Le aplicamos el algoritmo SVD quedándonos con 3 valores singulares
r = 3
U, Σ, V = SVD_algoritmo(M, r, 0.01)

print(f"A partir de la matriz de datos original M:\n\n{M}\n")
print("Obtenemos las siguientes matrices para la descomposición en valores singulares:")
print(f"Matriz U:\n\n{U}\n")
print(f"Matriz Σ:\n\n{Σ}\n")
print(f"Matriz V:\n\n{V}\n")
print("La descomposición SVD de la matriz M dada será M = U*Σ*V_transpuesta")

Comprobación de que los autovalores obtenidos con las matrices M * M_transpuesta y M_transpuesta * M son iguales
Autovalores de la matriz MU = M * M_transpuesta: [array([[476.10056425]]), array([[150.30780278]]), array([[67.18828518]])]
Autovalores de la matriz MV = M_transpuesta * M: [array([[476.10248024]]), array([[150.30233012]]), array([[67.1869512]])]

A partir de la matriz de datos original M:

[[3 0 0 7 4 7 8 0 0 0]
 [1 1 0 5 0 0 8 3 0 2]
 [0 2 2 6 6 1 1 0 5 0]
 [4 0 2 7 9 4 1 1 2 1]
 [0 0 2 2 0 0 8 4 5 6]]

Obtenemos las siguientes matrices para la descomposición en valores singulares:
Matriz U:

[[ 0.56967114  0.05012506 -0.66379539]
 [ 0.36764751 -0.4379006  -0.20554183]
 [ 0.37294228  0.31345708  0.5308659 ]
 [ 0.50985701  0.52722096  0.15968554]
 [ 0.37585901 -0.65537261  0.45805038]]

Matriz Σ:

[[21.81977269  0.          0.        ]
 [ 0.         12.25978508  0.        ]
 [ 0.          0.          8.19676468]]

Matriz V:

[[ 0.18849703  0.14638791 -0.19135706]
 [ 0.05109

## Algoritmo implementado sobre el tema de "Conjuntos de Ítems Frecuentes":
### - Algoritmo A-Priori

In [8]:
# He creado un fichero de ejemplo llamado "CestasCompra.txt" con diez listas de la compra,
# para probar el algoritmo.

In [9]:
# Abrimos el fichero donde tenemos almacenadas todas las cestas de la compra
with open('CestasCompra.txt', 'r') as f:
    lista_cestas_bruto = f.read().split('}{')

f.close()  # Cerramos el fichero f ya que su contenido lo hemos volcado en la lista "lista_cestas"

for i in range(len(lista_cestas_bruto)):
    lista_cestas_bruto[i] = lista_cestas_bruto[i].split(',')

print(lista_cestas_bruto)

# Vemos que dentro de "lista_cestas_bruto" hay 12 listas, cuando en realidad sabemos que tenemos diez listas
# de la compra. Es debido a que la lista de índice 0 es [''], y la lista de índice 11 es ['\n']. Para el
# algoritmo A-Priori nos interesa trabajar con las listas cuyos índices van de 1 a 10, es decir, todas menos la
# primera y la última.

lista_cestas = []

for i in range(1, len(lista_cestas_bruto) - 1):
    lista_cestas.append(lista_cestas_bruto[i])

[[''], ['mostaza', 'agua', 'salchichas', 'plátanos', 'jamón', 'pan'], ['zanahorias', 'alubias', 'cebollas', 'huevos'], ['chorizo', 'plátanos', 'salchichas', 'detergente', 'mostaza'], ['lavavajillas', 'langostinos', 'cerveza'], ['jamón', 'leche', 'pan', 'lechuga'], ['desodorante', 'dentífrico', 'mostaza', 'ketchup', 'salchichas', 'chocolate'], ['pan', 'leche', 'cerveza', 'pañales'], ['aceitunas', 'maíz', 'pan', 'detergente', 'leche', 'macarrones'], ['plátanos', 'chorizo', 'langostinos', 'leche', 'pan', 'chocolate'], ['alubias', 'maíz', 'cerveza', 'jamón', 'pan', 'plátanos'], ['\n']]


In [10]:
# Función que implementa el algoritmo A-Priori
# Debemos haber importado la librería Numpy con anterioridad.

def APriori_algoritmo(lista_cestas, s):
    """Función que implementa el método A-Priori para la búsqueda de pares de ítems frecuentes en cestas
    de la compra.
    
    Parameters
    ----------
    lista_cestas : Lista que contiene todas las cestas de la compra que vamos a analizar. Cada cesta de la compra
                   será una lista de artículos dentro de esa lista.
    
    s            : Umbral de soporte requerido. Los pares de ítems para que sean considerados frecuentes deberán
                   aparecer en al menos s cestas de la compra diferentes.
                   Lo mismo para los ítems por separado, para que sean considerados frecuentes deberán aparecer
                   en al menos s cestas de la compra diferentes.
                   Un s típico sería el 1% de las cestas.
    
    Returns
    -------
    pares_frecuentes : Devuelve una lista con los pares de artículos frecuentes en las cestas de la compra
                       analizadas.
    """
    
    # ---------------------- Primera pasada del algoritmo A-Priori --------------------------------
    
    # Vamos a crear primeramente la tabla hash que traduce los nombres de los artículos de las cestas de la
    # compra a números enteros de 1 a n, siendo n el número de artículos diferentes que tenemos.
    
    tabla_hash = {}  # Será un diccionario donde las claves serán los nombres de los artículos y los valores
                     # los números enteros que representarán a cada artículo.
                     # Esta representación de los artículos mediante enteros en lugar de como cadenas de
                     # caracteres se hace para que ocupe menos memoria la representación de dichos artículos
                     # en las tablas de contadores sucesivas.
    contadores_articulos = []  # Inicializamos esta lista que será el array de contadores de cada uno de los
                               # artículos en las cestas de la compra.
    
    for i in range(len(lista_cestas)):
        for j in range(len(lista_cestas[i])):
            if lista_cestas[i][j] not in tabla_hash:
                tabla_hash[lista_cestas[i][j]] = len(tabla_hash) + 1
                contadores_articulos.append([tabla_hash.get(lista_cestas[i][j]), 1])
            else:
                contadores_articulos[tabla_hash.get(lista_cestas[i][j]) - 1][1] += 1  # Usamos ahí como primer
                                                            # índice de la lista "contadores_articulos" el valor
                                                            # [tabla_hash.get(lista_cestas[i][j]) - 1], restando
                                                            # ese 1 porque los índices de una tabla empiezan
                                                            # a contar desde 0.
    
    # Vamos a examinar los artículos que son frecuentes comos conjuntos unitarios. Se considera que son
    # frecuentes si aparecen en un número de cestas mayor que el umbral de soporte s.
    # Creamos una nueva numeración de 1 a m sólo para identificar los artículos que son frecuentes. Sustituimos
    # la segunda columna del array "contadores_articulos", la que contiene los contadores de los artículos,
    # por la nueva numeración. El array ahora sigue teniendo dos columnas, la primera columna contiene el índice
    # del artículo de 1 a n, y la segunda el índice de artículo frecuente de 1 a m, ó 0 si no es frecuente.
    # Al hacer eso conseguimos ahorrar memoria al no tener que almacenar la columna con los contadores de los
    # artículos, que era de longitud n, ya que no la necesitaremos más.
    
    # NOTA: Otra opción hubiera sido añadir esa nueva numeración al array en una columna nueva, una tercera
    # columna. Esto tendría sentido si posteriormente necesitáramos utilizar los valores de los contadores de
    # los artículos individuales, ya que aumenta el espacio de memoria necesario para almacenar el array.
    
    m = 0  # Variable auxiliar para asignar la nueva numeración a los artículos frecuentes.
    
    for n in range(len(contadores_articulos)):
        if contadores_articulos[n][1] >= s:
            m += 1
            contadores_articulos[n][1] = m
        else:
            contadores_articulos[n][1] = 0
    
    # ---------------------- Segunda pasada del algoritmo A-Priori --------------------------------
    
    # Vamos a generar y contar ahora todos los conjuntos de pares en los que ambos artículos son frecuentes.
    
    # A partir del conjunto de artículos frecuentes, que es de tamaño m, el máximo número de pares de artículos
    # frecuentes diferentes que podemos conseguir es (m*(m-1))/2
    
    contadores_pares = np.zeros(((m*(m-1))//2, 2), dtype=int)  # Creamos un array donde almacenaremos los
                                                # contadores de cada uno de los pares de artículos
                                                # frecuentes, y los inicializamos a cero. Esos
                                                # contadores serán la segunda columna del array.
    # En la primera columna del array guardamos el índice que corresponde con cada par según lo asignamos
    # por el método de la matriz triangular al almacenar los pares de artículos frecuentes usando una
    # matriz triangular unidimensional
    for i in range(contadores_pares.shape[0]):
        contadores_pares[i, 0] = i+1
    
    # Para cada cesta, comenzamos mirando la tabla de artículos frecuentes para ver cuáles de sus artículos
    # son frecuentes
    for i in range(len(lista_cestas)):
        frecuentes_cestas = []  # Lista para almacenar los artículos frecuentes de cada cesta.
        for j in range(len(lista_cestas[i])):
            if contadores_articulos[tabla_hash[lista_cestas[i][j]] - 1][1] != 0:
                frecuentes_cestas.append(contadores_articulos[tabla_hash[lista_cestas[i][j]] - 1])  # Almacenamos
                                               # para cada cesta, en la lista "frecuentes_cestas", los artículos
                                               # de dicha cesta que son frecuentes, en la forma [n, m], siendo
                                               # n el índice del artículo sobre el total de artículos, y m el
                                               # índice del artículo sobre el total de artículos frecuentes.
        
        if len(frecuentes_cestas) > 1:  # Si no es mayor que 1 no podemos hacer ningún par de artículos frecuentes
                                        # con esa cesta.
            
            # Creamos una nueva lista donde almacenaremos los índides de los artículos frecuentes de cada cesta
            # en orden de menor a mayor valor de índice, para poder aplicar el método de la matriz triangular
            indices = []
            for v in range(len(frecuentes_cestas)):
                indices.append(frecuentes_cestas[v][1])
            indices.sort()
            
            # Método de la matriz triangular
            for l in range(len(indices) - 1):
                for p in range(l+1, len(indices)):
                    # Utilizamos el método de la matriz triangular para almacenar los pares de artículos frecuentes
                    # Como ya he dicho, nos quedamos con los índices de artículos frecuentes (de 1 a m), para
                    # crear los nuevos índices k.
                    indice1 = indices[l]
                    indice2 = indices[p]
                    k = (indice1 - 1) * (m - indice1/2) + indice2 - indice1
                    # k es el índice para utilizar una matriz triangular unidimensional. Empieza a contar en 1.
                    contadores_pares[int(k)-1, 1] += 1
    
    # Obtenemos que los pares de artículos frecuentes en las cestas de la compra analizadas son los siguientes:
    
    pares_frecuentes = []  # Lista donde almacenaremos los pares de artículos frecuentes encontrados en las
                           # cestas de la compra analizadas.
    
    for i in range(contadores_pares.shape[0]):
        if contadores_pares[i, 1] >= s:
            pares_frecuentes.append(contadores_pares[i])
    
    return pares_frecuentes
    
    # Con lo que he programado hasta aquí, los pares los devuelve en la forma [k, num], siendo k el índice del
    # par de artículos según la codificación en la matriz triangular unidimensional, y num el número de
    # apariciones de dicho par en el conjunto de cestas de la compra analizado.
    # Faltaría pasar de dicho índice k, a los índices de artículos frecuentes, y de esos índices a los índices
    # de artículos totales, mediante la tabla "contadores_articulos", y por último a los nombres de los artículos
    # mediante la "tabla_hash".

In [11]:
# Ejemplo de uso del algoritmo A-Priori

# Vamos a calcular los pares de artículos frecuentes que hay en el fichero que he creado de
# ejemplo "CestasCompra.txt". Ya hemos guardado su contenido en la lista "lista_cestas" con anterioridad.
# Utilizaremos para el ejemplo un umbral de soporte igual a 3
s = 3
pares = APriori_algoritmo(lista_cestas, s)
print(pares)

# Como vemos salen para ese valor del umbral de soporte igual a 3, que tenemos cuatro pares de artículos
# frecuentes. Si hacemos el recorrido inverso de los índices vemos que se corresponden con los siguientes:
#     {mostaza, salchichas} que aparece 3 veces en el conjunto de cestas de la compra.
#     {plátanos, pan} que aparece 3 veces en el conjunto de cestas de la compra.
#     {jamón, pan} que aparece 3 veces en el conjunto de cestas de la compra.
#     {pan, leche} que aparece 4 veces en el conjunto de cestas de la compra.

[array([1, 3]), array([13,  3]), array([16,  3]), array([20,  4])]
